In [1]:
import os
import pandas as pd

from tqdm import tqdm
from transformers import pipeline, set_seed
from transformers import BioGptTokenizer, BioGptForCausalLM
from aug.gpt import *
import warnings
warnings.filterwarnings("ignore")
MIMIC_EYE_PATH = "F:\\mimic-eye"

In [2]:
lesion = "Atelectasis"
questions = [
    f"What is {lesion}?",
    f"What are the symptoms associated with {lesion}?",
    f"What can cause {lesion}?",
    f"What are the patient’s symptoms that are relevant for {lesion}?",
    f"What are the relevant clinical signs for the etiological diagnosis of {lesion}?",
    f"What are the relevant laboratory data for the etiological diagnosis of {lesion}?",
    f"What are the relevant clinical characteristics for the etiological diagnosis of {lesion}",
    f"What are the patient’s personal relevant history for the etiological diagnosis of {lesion}",
]

In [3]:
from transformers import AutoTokenizer, AutoModel, DistilBertForMaskedLM, BertForMaskedLM, LlamaForCausalLM, LlamaTokenizer, LlamaForCausalLM
import torch

In [4]:
'''
LLaMa2
'''

# from transformers import (
#     AutoTokenizer,
#     AutoModel,
#     DistilBertForMaskedLM,
#     BertForMaskedLM,
#     LlamaTokenizer,
#     LlamaForCausalLM,
# )
# import torch
# from secret import *

# tokenizer = LlamaTokenizer.from_pretrained(
#     "meta-llama/Llama-2-7b-hf", token=HUGGING_FACE_TOKEN
# )
# model = LlamaForCausalLM.from_pretrained(
#     "meta-llama/Llama-2-7b-hf",
#     token=HUGGING_FACE_TOKEN,
#     device_map="auto",
#     torch_dtype=torch.float16,
# )  # )
# # model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
# generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
# set_seed(0)

'\nLLaMa2\n'

In [5]:
'''
Mistral
'''

# generator = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1", torch_dtype=torch.bfloat16, device_map="auto", pad_token_id=2, eos_token_id=2)

'\nMistral\n'

In [6]:
# # do greedy
# res_dict = {}
# for q in questions:
#     res = generator(f"""Question: {q} 
#                     Answer:""",  num_return_sequences=1, max_length=512)
#     res_dict[q] = res[0]['generated_text']

In [7]:
import openai
from secret import *
openai.api_key = OPENAI_API_KEY

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    retry_if_exception_type
)  # for exponential backoff


@retry(
    retry=retry_if_exception_type((openai.error.APIError, openai.error.APIConnectionError, openai.error.RateLimitError, openai.error.ServiceUnavailableError, openai.error.Timeout)), 
    wait=wait_random_exponential(multiplier=1, max=60), 
    stop=stop_after_attempt(10)
)
def chat_completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)


In [8]:
res_dict = {}
for q in questions:
    res = chat_completion_with_backoff(
                # model="gpt-3.5-turbo",
                model="gpt-4",
                messages=[
                    # {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "system", "content": "You are an experienced radiologist. Use a keyword-based report to answer the following questions."},
                    {"role": "user", "content": f"{q}"},
                ],
                temperature=0.2,
                n=1,
            )
    res_dict[q] = res

In [21]:
# post-process res_dict
processed_res_dict = {q: res_dict[q]['choices'][0]['message']['content'] for q in questions}

In [22]:
processed_res_dict

{'What is Atelectasis?': 'Atelectasis refers to the partial or complete collapse of a lung or a section (lobe) of a lung. It is typically caused by a blockage of the tiny airways (bronchioles) or by pressure on the outside of the lung. Atelectasis can lead to decreased oxygen levels in the blood, which can be serious if not treated promptly. Symptoms may include difficulty breathing, cough, and chest pain.',
 'What are the symptoms associated with Atelectasis?': 'As a radiologist, I interpret imaging rather than directly dealing with symptoms. However, generally, symptoms of Atelectasis may include difficulty breathing, rapid shallow breathing, wheezing, coughing, and chest pain. Please consult with a healthcare provider for a comprehensive understanding.',
 'What can cause Atelectasis?': '1. Airway Obstruction: This could be due to a tumor, foreign body, mucus plug, or severe asthma.\n2. Compression: Conditions such as pleural effusion, pneumothorax, or a large tumor can compress the 

In [24]:
for i, q in enumerate(questions):
    print(i+1)
    print(processed_res_dict[q])

1
Atelectasis refers to the partial or complete collapse of a lung or a section (lobe) of a lung. It is typically caused by a blockage of the tiny airways (bronchioles) or by pressure on the outside of the lung. Atelectasis can lead to decreased oxygen levels in the blood, which can be serious if not treated promptly. Symptoms may include difficulty breathing, cough, and chest pain.
2
As a radiologist, I interpret imaging rather than directly dealing with symptoms. However, generally, symptoms of Atelectasis may include difficulty breathing, rapid shallow breathing, wheezing, coughing, and chest pain. Please consult with a healthcare provider for a comprehensive understanding.
3
1. Airway Obstruction: This could be due to a tumor, foreign body, mucus plug, or severe asthma.
2. Compression: Conditions such as pleural effusion, pneumothorax, or a large tumor can compress the lung and cause atelectasis.
3. Postoperative: This is common after surgery, especially in the upper abdomen or che

In [12]:
# Mistral
# generator = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1", torch_dtype=torch.bfloat16, device_map="auto", pad_token_id=2, eos_token_id=2)
# set_seed(0)
# generator("The average blood pressure for human is ",  max_new_tokens=10, num_return_sequences=1, do_sample=True)

In [13]:
# import openai
# from secret import *
# openai.api_key = OPENAI_API_KEY

# from tenacity import (
#     retry,
#     stop_after_attempt,
#     wait_random_exponential,
#     retry_if_exception_type
# )  # for exponential backoff


# @retry(
#     retry=retry_if_exception_type((openai.error.APIError, openai.error.APIConnectionError, openai.error.RateLimitError, openai.error.ServiceUnavailableError, openai.error.Timeout)), 
#     wait=wait_random_exponential(multiplier=1, max=60), 
#     stop=stop_after_attempt(10)
# )
# def chat_completion_with_backoff(**kwargs):
#     return openai.ChatCompletion.create(**kwargs)


# res = chat_completion_with_backoff(
#                 # model="gpt-3.5-turbo",
#                 model="gpt-4",
#                 messages=[
#                     # {"role": "system", "content": "You are a helpful assistant."},
#                     {"role": "system", "content": "You are a medical expert predicting the possible value of clinical features."},
#                     {"role": "user", "content": f"What can be the next word for the following context (please only return one numerical word):\n\n {prompt}"},
#                 ],
#                 temperature=0.2,
#                 n=50
#             )

#             res = [c['message']['content'] for c in res['choices']]